# Exploratory notebook

## Problem formulation

Data set creation

- Use tweets from the top 5 politicians by vote from each party with at least 5 mandates as of the 2022 election.

- The gathered tweets will be restricted by date, e.g. no tweets older than 2 years

Problem ideas

- Create a candidacy test based on the politicians' tweets

- Cluster politicians based on the words used in their tweets - do these clusters match their parties?

- Can we recognize some "mærkesager" through the tweets

Ideas for creating a solution

- A recommender system is created based on a tweet and the tweets of politicians to form a candidacy test. Here similar items can be used to obtain a similarity score used by the recommender system.

- Different clustering techniques taught in the course will be used to cluster the politicians

- Word clouds for each party/politician reveal the words they used the most and using frequent itemsets we may recognize a "mærkesag"

## Twitter ids

Here's a list of the twitter id for the top 3 candidates from each of the parties with at least 5 "mandater" in "folketinget" after the 2022 election.

The list is based on  https://www.dr.dk/nyheder/politik/folketingsvalg/valgte

In [ ]:
from Data.twitter_ids import twitter_ids

for parti in twitter_ids:
    for person in twitter_ids[parti]:
        print(person)
        print(twitter_ids[parti][person])

magnus_heunicke
22695562
nicolai_wammen
2803948786
mattias_tesfaye
546254893
jakob_ellemann
155584627
soren_gade
975064362359623680
sophie_lohde
44611200
lars_lokke
26201346
henrik_frandsen
1249019841924734977
rosa_eriksen
1560192117858861056
jacob_mark
2373406198
pia_dyhr
65025162
kirsten_andersen
235646319
dennis_flydtkjær
531595033
peter_skaarup
3144074691
soren_espersen
2444718215
alex_vanopslagh
1531564633
ole_olesen
2222188479
solbjorg_jakobsen
1548641745264644099
soren_pape
2712091824
mette_abildgaard
37877392
rasmus_jarlov
1225930531
pelle_dragsted
119879630
mai_villadsen
4724782641
rosa_lund
736979161
martin_lidegaard
1070745218
samira_nawa
92107029
katrine_robsoe
2491403660
pernille_vermund
24687777
lars_mathiesen
980721900
kim_andersen
783935815600799744
franciska_rosenkilde
777113466205274112
christina_olumeko
1324801335372488707
torsten_gejl
2806864609
morten_messerschmidt
509288627
pia_kjarsgaard
1054640354690039809
peter_kofod
1613378210


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=30797f9c-952e-45b4-98d4-31c9ac73ae78' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>